In [7]:
import requests
import pandas as pd
import plotly.express as px
from dotenv import load_dotenv   #for python-dotenv method
load_dotenv()                    #for python-dotenv method
import os
liste_villes = ["Paris","Londres","New York","Ibiza","Vienne","Valence","Lisbonne","Shanghai","Casablanca","Nairobi","Los Angeles", "Tokyo", "Rio de Janeiro", "Fort-de-France"]


COORDONNEES D'UNE SEULE VILLE

In [8]:
# ESSAI MANUEL POUR UNE SEULE VILLE 

url = "https://nominatim.openstreetmap.org/search?"
headers = {"accept": "application/json", 'user_agent':'charles.mittempergher@gmail.com'}
p = {'city' : 'Toulouse', 'country':'France', 'format':'json', 'limit':1}

response = requests.get(url, headers=headers, params=p)

BOUCLE POUR LES COORDONNEES TOUTES LES VILLES

In [9]:
# RECUPERATION DES DONNEES POUR TOUTE LA LISTE DES VILLES
url = "https://nominatim.openstreetmap.org/search?"
headers = {'user_agent':'charles.mittempergher@gmail.com'}

df_villes = pd.DataFrame(columns=['ville','lat','lon'])

for i in range(len(liste_villes)):
    
    p = {'city' : {liste_villes[i]}, 'format':'json', 'limit':1, 'email':'charles@gmail.com'}

    r = requests.get(url, headers=headers, params=p)
    ville = liste_villes[i]
    lat   = r.json()[0]['lat']
    lon   = r.json()[0]['lon']

    df_villes.loc[i] = [ville, lat,lon]
    print(f'Ajout de {liste_villes[i]} effectué')

Ajout de Paris effectué
Ajout de Londres effectué
Ajout de New York effectué
Ajout de Ibiza effectué
Ajout de Vienne effectué
Ajout de Valence effectué
Ajout de Lisbonne effectué
Ajout de Shanghai effectué
Ajout de Casablanca effectué
Ajout de Nairobi effectué
Ajout de Los Angeles effectué
Ajout de Tokyo effectué
Ajout de Rio de Janeiro effectué
Ajout de Fort-de-France effectué


In [10]:
df_villes.head()

,ville,lat,lon
0,Paris,48.8588897,2.3200410217200766
1,Londres,51.5074456,-0.1277653
2,New York,40.7127281,-74.0060152
3,Ibiza,38.974390099999994,1.4197463178515335
4,Vienne,48.2083537,16.3725042


 DEBUT DE LA PARTIE AVEC OPENWEATHER

In [11]:
url_ow = "https://api.openweathermap.org/data/2.5/forecast?"

lat = 43.2961743
lon = 5.3699525
p = {'lat' : lat, 'lon' : lon, 'units':'metric', 'appid' :  os.environ.get('openweather_active')}
response = requests.get(url_ow, params=p)


In [12]:
response.json().get('list')

[{'dt': 1731693600,
  'main': {'temp': 14.42,
   'feels_like': 13.72,
   'temp_min': 14.42,
   'temp_max': 14.43,
   'pressure': 1022,
   'sea_level': 1022,
   'grnd_level': 1015,
   'humidity': 69,
   'temp_kf': -0.01},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01n'}],
  'clouds': {'all': 0},
  'wind': {'speed': 1.57, 'deg': 185, 'gust': 1.62},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2024-11-15 18:00:00'},
 {'dt': 1731704400,
  'main': {'temp': 14.59,
   'feels_like': 13.8,
   'temp_min': 14.59,
   'temp_max': 14.68,
   'pressure': 1023,
   'sea_level': 1023,
   'grnd_level': 1016,
   'humidity': 65,
   'temp_kf': -0.09},
  'weather': [{'id': 800,
    'main': 'Clear',
    'description': 'clear sky',
    'icon': '01n'}],
  'clouds': {'all': 3},
  'wind': {'speed': 1.31, 'deg': 214, 'gust': 1.72},
  'visibility': 10000,
  'pop': 0,
  'sys': {'pod': 'n'},
  'dt_txt': '2024-11-15 21:00:00'},
 {'dt': 173171520

CREATION DU DATAFRAME AVEC LES PREVISIONS METEO POUR UNE SEULE VILLE

In [13]:
liste_previsions_une_ville = response.json().get('list')

len(liste_previsions_une_ville)
df_prevision_une_ville = pd.DataFrame(columns=['Ville','date','heure', 'temp','id_type_temps','type_temps'])
for i in range(len(liste_previsions_une_ville)):
    
    ville = 'Marseille'
    dt_text= liste_previsions_une_ville[i]['dt_txt']
    date = dt_text.split(" ")[0]
    heure= dt_text.split(" ")[1]
    temp=liste_previsions_une_ville[i]['main']['temp']
    id_type_temps=liste_previsions_une_ville[i]['weather'][0]['id']
    type_temps=liste_previsions_une_ville[i]['weather'][0]['main']

    df_prevision_une_ville.loc[i] = [ville,date,heure,temp,id_type_temps,type_temps]

In [14]:
df_prevision_une_ville.head()

,Ville,date,heure,temp,id_type_temps,type_temps
0,Marseille,2024-11-15,18:00:00,14.42,800,Clear
1,Marseille,2024-11-15,21:00:00,14.59,800,Clear
2,Marseille,2024-11-16,00:00:00,14.34,800,Clear
3,Marseille,2024-11-16,03:00:00,13.87,800,Clear
4,Marseille,2024-11-16,06:00:00,13.42,800,Clear


ON PREND PAS LES PREVISIONS ENTRE MINUIT ET 6H

In [15]:
mask = df_prevision_une_ville['heure'] >= '06:00:00'
data = df_prevision_une_ville[mask]

In [16]:
data.head()

,Ville,date,heure,temp,id_type_temps,type_temps
0,Marseille,2024-11-15,18:00:00,14.42,800,Clear
1,Marseille,2024-11-15,21:00:00,14.59,800,Clear
4,Marseille,2024-11-16,06:00:00,13.42,800,Clear
5,Marseille,2024-11-16,09:00:00,14.33,800,Clear
6,Marseille,2024-11-16,12:00:00,15.70,800,Clear


+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

PASSAGE AUX CHOSES SERIEUSES

BOUCLE POUR TOUTES LES PREVISIONS DE TOUTES LES VILLES

In [17]:
url_ow = "https://api.openweathermap.org/data/2.5/forecast?"

df_prevision_toutes_les_villes = pd.DataFrame(columns=['Ville','date','heure','Part_of_day', 'temp','id_type_temps','type_temps','couv_nuageuse','vent','prob_pluie','pluie'])

# BOUCLE SUR LES VILLES 
for v in range(len(df_villes)):

    lat = df_villes.loc[v,"lat"]
    lon = df_villes.loc[v,"lon"]
    p = {'lat' : lat, 'lon' : lon, 'units':'metric', 'appid' :  os.environ.get('openweather_active')}
    response = requests.get(url_ow, params=p)
    response.json().get('list')

    liste_previsions_une_ville_final = response.json().get('list')
    nb_prev = len(liste_previsions_une_ville_final)
    # BOUCLES SUR LES PREVISIONS
    for i in range(len(liste_previsions_une_ville_final)):
        
        ville = df_villes.loc[v,"ville"]
        dt_text= liste_previsions_une_ville_final[i]['dt_txt']
        date = dt_text.split(" ")[0]
        heure= dt_text.split(" ")[1]
        temp=liste_previsions_une_ville_final[i]['main']['temp']
        id_type_temps=liste_previsions_une_ville_final[i]['weather'][0]['id']
        type_temps=liste_previsions_une_ville_final[i]['weather'][0]['main']
        couv_nuageuse=liste_previsions_une_ville_final[i]['clouds']['all']
        vent=liste_previsions_une_ville_final[i]['wind']['speed']
        prob_pluie=liste_previsions_une_ville_final[i]['pop']
        pluie = 0
        if 'rain' in liste_previsions_une_ville_final[i].keys():
            pluie = liste_previsions_une_ville_final[i]['rain']['3h']
        pod = liste_previsions_une_ville_final[i]['sys']['pod']

        df_prevision_toutes_les_villes.loc[len(df_prevision_toutes_les_villes)+1] = [ville,date,heure,pod,temp,id_type_temps,type_temps,couv_nuageuse,vent,prob_pluie,pluie]
        

In [18]:
mask = df_prevision_toutes_les_villes['heure'] >= '06:00:00'
data_toutes_les_villes = df_prevision_toutes_les_villes[mask]

ON REGARDE LES DIFFERENTES VALEURS DU TYPE DE METEO POUR ETABLIR UN CLASSEMENT

In [19]:
data_toutes_les_villes['id_type_temps'].unique()

array([803, 804, 801, 500, 501, 802, 800, 502], dtype=int64)

MAPPING ET CREATION D'UNE NOUVELLE COLONNE AVEC LA NOTE DE LA PREVISION

In [20]:
classement = {800: 10, 801:9,802:8,803:7,804:6,500:4,501:3, 601:2}
data_toutes_les_villes['note_type_temps']=data_toutes_les_villes['id_type_temps'].apply(lambda x: classement[x] if x in classement else 1)

C:\Users\Admin\AppData\Local\Temp\ipykernel_12804\3510448894.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_toutes_les_villes['note_type_temps']=data_toutes_les_villes['id_type_temps'].apply(lambda x: classement[x] if x in classement else 1)


In [21]:
data_toutes_les_villes.tail()

,Ville,date,heure,Part_of_day,temp,id_type_temps,type_temps,couv_nuageuse,vent,prob_pluie,pluie,note_type_temps
554,Fort-de-France,2024-11-19,21:00:00,d,27.55,802,Clouds,26,3.80,0.00,0.00,8
557,Fort-de-France,2024-11-20,06:00:00,n,25.16,500,Rain,15,3.01,0.97,0.15,4
558,Fort-de-France,2024-11-20,09:00:00,n,25.06,801,Clouds,15,2.52,0.00,0.00,9
559,Fort-de-France,2024-11-20,12:00:00,d,26.94,500,Rain,23,2.77,0.20,0.21,4
560,Fort-de-France,2024-11-20,15:00:00,d,28.38,500,Rain,91,3.28,1.00,0.96,4


In [22]:
total_par_ville = data_toutes_les_villes.groupby('Ville')['note_type_temps'].sum()

In [23]:
total_par_ville

Ville
Casablanca        218
Fort-de-France    157
Ibiza             238
Lisbonne          227
Londres           188
Los Angeles       261
Nairobi           174
New York          247
Paris             176
Rio de Janeiro    202
Shanghai          179
Tokyo             177
Valence           224
Vienne            215
Name: note_type_temps, dtype: int64

In [24]:
moyenne_temp_par_ville = data_toutes_les_villes.groupby('Ville')['temp'].mean()

ON FUSIONNE LES TROIS TABLES POUR AVOIR NOS DONNEES FINALES (VILLES ET COORDONNEES, SCORE, TEMPERATURE MOYENNE)

In [25]:
df_final = df_villes.merge(total_par_ville,left_on='ville',right_on='Ville')
df_final = df_final.merge(moyenne_temp_par_ville,left_on='ville', right_on='Ville')

CHANGEMENT DU TYPE DE DONNEES POUR LATITUDE ET LONGITUDE

In [26]:
df_final['lat'] = df_final['lat'].astype('float64')
df_final['lon'] = df_final['lon'].astype('float64')

AFFICHAGE DU DATASET FINAL UTILISE POUR LA CARTE

In [27]:
df_final.sort_values("note_type_temps", ascending=False)

,ville,lat,lon,note_type_temps,temp
10,Los Angeles,34.053691,-118.242766,261,14.532667
2,New York,40.712728,-74.006015,247,11.178000
3,Ibiza,38.974390,1.419746,238,17.684667
6,Lisbonne,38.707751,-9.136592,227,17.077000
5,Valence,39.469707,-0.376335,224,17.548333
8,Casablanca,33.594514,-7.620028,218,20.971333
4,Vienne,48.208354,16.372504,215,6.558667
12,Rio de Janeiro,-22.911014,-43.209373,202,24.290000
1,Londres,51.507446,-0.127765,188,6.773667
7,Shanghai,31.232344,121.469102,179,15.497000


CREATION DE LA CARTE

In [28]:
fig = px.scatter_mapbox(df_final,lat="lat", lon="lon", color="note_type_temps", hover_name="ville", hover_data='temp', height=700,
mapbox_style="open-street-map",title='Où on part ce weekend ?',zoom=1,color_continuous_scale=["red","orange","yellowgreen","green"], size="temp")
fig.update_layout(title_x=0.5)
fig.show()